# NOTEBOOK USED FOR PRACTICAL ANALYSIS

## Authors: Dani Gómez, Ricard Tarré




# 1.Code for two-D reconstruction error

#### Parameters of the function

This piece of code, receives as an entrance four parameters:

    1. K: Number of breaks
    2. Path of dataset
    3. Kernel function
    4. Set of parameters
    
#### Purpose of the code
    
For every set of parameters it computes the reconstruction error that would be achieven if we apply kernel PCA with the kernel function chosen to the dataset given. 
##### !!!The dataset must be two dimensional!!!

#### What is the process?

For every set of parameters,it breaks the datasets in K groups and we iterate for each group. In every iteration one of the groups is left as 'test group' and the others are used to compute kernel matrix and corresponding projected vectors. Then we compute reconstruction error for every point in the test group and we store the average of all the points. For every group we store the reconstrcution error and we finally show to the user the average error of all the groups. 

#### Result

The user will see in the screen the reconstruction error for every combination of parameters, that have been applied with the kernel function chosen.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from scipy.linalg import eigh
import random

from sklearn.model_selection import KFold
from sklearn.decomposition import KernelPCA

################### KERNELS ##########################
def Polynomic(x,z,d,c):
    return pow(np.dot(x,z)+c,d)

def Gaussian(x,z,sigma):
    return np.exp((-(np.linalg.norm(x-z)**2))/(2*sigma**2))

def Sigmoid(x,z,aplha,c):
    return np.tanh(aplha*np.dot(x,z)+c)
#######################################################


################### KERNEL MATRIXES ###################
def computeKernelMatrix(kernel, X, parameters):
    if kernel == 'rbf':
        return GaussianMatrix(X,parameters['sigma'])
    elif kernel == 'sigmoid':
        return SigmoidMatrix(X,parameters['alpha'],parameters['c'])
    elif kernel == 'polynomial':
        return PolynomicalMatrix(X,parameters['d'],parameters['c'])

def GaussianMatrix(X,sigma):
    row,col=X.shape
    GassMatrix=np.zeros(shape=(row,row))
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            GassMatrix[i,j]=Gaussian(v_i.T,v_j.T,sigma)
            j+=1
        i+=1
    return GassMatrix

def SigmoidMatrix(X,sigma, alpha, c):
    row,col=X.shape
    SigmoidMatrix=np.zeros(shape=(row,row))
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            SigmoidMatrix[i,j]=Sigmoid(v_i.T,v_j.T,alpha, c)
            j+=1
        i+=1
    return SigmoidMatrix

def PolynomicalMatrix(X,d,c):
    row,col=X.shape
    PolMatrix=np.zeros(shape=(row,row))
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            PolMatrix[i,j]=Polynomic(v_i.T,v_j.T,d,c)
            j+=1
        i+=1
    return PolMatrix
#####################################################


################### INVERSES ########################
def vPolynomic(intr,d,c):
    return pow(intr,(1/d))-c

def vSigmoid(intr,alpha,c):
    return (np.arctanh(intr)-c)/alpha
#####################################################


################### PRE-IMAGES ######################
def ComputePreImage(kernel, parameters, val, evec, X, gamma):
    if kernel == 'rbf':
        return ComputePreImageRBF(X,val,evec, gamma, parameters['sigma'])
    elif kernel == 'sigmoid':
        return ComputePreImageSigmoid(X,val,evec, gamma, parameters['alpha'],parameters['c'])
    elif kernel == 'polynomial':
        return ComputePreImagePolynomic(X,val,evec, gamma, parameters['d'],parameters['c'])
    
def ComputePreImageRBF(X,val,evec, gamma, sigma):
    X_t = np.ones(len(X[0]))
    new_X_t = iterateRBF(X_t,X,val,evec,gamma, sigma) #1500
    i = 0
    k = 0
    while np.linalg.norm(new_X_t - X_t) > 0.001 and i < 500:
        X_t = new_X_t
        i += 1
        if i == 500 and k < 3:
            i = 0
            k += 1
            li = []
            for i in range(len(X[0])):
                li.append(random.uniform(-1, 1)) # If not converges, initialize random.
            X_t = np.array(li) 
        new_X_t = iterateRBF(X_t,X,val,evec,gamma, sigma)
    return new_X_t

def ComputePreImagePolynomic(X,val,evec, gamma, d, c):
    if d%2 == 0:
        return ComputePreImagePolynomic_even(X, val, evec, gamma, d, c)
    else:
        return ComputePreImagePolynomic_odd(X, val, evec, gamma, d, c)

def ComputePreImagePolynomic_odd(X,val,evec, gamma,d,c):
    e = np.array([[1.0,0.0],[0.0,1.0]])
    preimage = np.array([0.0,0.0])
    for j in range(len(e)):
        intr = 0
        for i in range(len(X)):
            intr += gamma[i]*Polynomic(e[j],X[i],d,c)
        if intr < 0:
            intr = -intr
        preimage += vPolynomic(intr,d,c)*e[j]
    return preimage
        
def ComputePreImagePolynomic_even(X,val,evec, gamma, d, c):
    X_t = np.ones(len(X[0]))
    new_X_t = iteratePolynomic(X_t,X,val,evec,gamma, d, c)
    i = 0
    while np.linalg.norm(new_X_t - X_t) > 0.001 and i < 1000:
        X_t = new_X_t
        new_X_t = iteratePolynomic(X_t,X,val,evec,gamma, d, c)
        i += 1
    return new_X_t
        
def ComputePreImageSigmoid(X,val,evec, gamma, alpha, c):
    e = np.array([[1,0],[0,1]])
    preimage = np.array([0,0])
    for j in range(len(e)):
        intr = 0
        for i in range(len(X)):
            intr += gamma[i]*Sigmoid(e[j],X[i],d,c)
        preimage += vSigmoid(intr,d,c)*e[j]
    return preimage

def iterateRBF(X_t,X,val,evec,gamma, sigma):
    num = 0
    den = 0 
    for i in range(len(X)):
        num += gamma[i]*Gaussian(X[i],X_t,sigma)*X[i]
        den += gamma[i]*Gaussian(X[i],X_t,sigma)
        if den == 0.0:
            return 0
    return num/den

def iteratePolynomic(X_t,X,val,evec,gamma, d, c):
    result = 0
    for i in range(len(X)):
        result += gamma[i]*pow((np.dot(X_t,X[i])+c)/(np.dot(X_t,X_t)+c),d-1)*X[i]
    return result
###################################################


################### GAMMA's ######################
def ComputeGamma(kernel, parameters, val, evec, point, X):
    if kernel == 'rbf':
        return ComputeGammaRBF(point, X, val, evec, parameters['sigma'])
    elif kernel == 'sigmoid':
        return ComputeGammaSigmoid(point, X, val, evec, parameters['alpha'],parameters['c'])
    elif kernel == 'polynomial':
        return ComputeGammaPolynomic(point, X, val, evec,parameters['d'],parameters['c'])
    
def ComputeGammaRBF(X_test, X, val, evec, sigma):
    gamma = []
    for i in range(len(X)):
        gamma.append(compute_gamma_RBF_i(i,X_test,X,val,evec,sigma))
    return gamma

def ComputeGammaPolynomic(X_test, X, val, evec, d, c):
    gamma = []
    for i in range(len(X)):
        gamma.append(compute_gamma_Polynomic_i(i,X_test,X,val,evec, d, c))
    return gamma

def ComputeGammaSigmoid(X_test, X, val, evec, aplha, c):
    gamma = []
    for i in range(len(X)):
        gamma.append(compute_gamma_Sigmoid_i(i,X_test,X,val,evec, aplha, c))
    return gamma
    
def compute_gamma_RBF_i(i,X_test,X,val,evec, sigma):
    gamma = 0
    for j in range(2):
        if j == 2:
            if max(val[:pj]) > max(val[pj+1:]):
                pj = np.argmax(val[:pj])
            else:
                pj = np.argmax(val[pj+1:])       
        pj = np.argmax(val)
        alphaj = evec[pj]
        for h in range(len(alphaj)):
            gamma += alphaj[i]*Gaussian(X[h],X_test,sigma)*alphaj[h]
    return gamma  

def compute_gamma_Polynomic_i(i,X_test,X,val,evec, d, c):
    gamma = 0
    for j in range(2):
        if j == 2:
            if max(val[:pj]) > max(val[pj+1:]):
                pj = np.argmax(val[:pj])
            else:
                pj = np.argmax(val[pj+1:])       
        pj = np.argmax(val)
        alphaj = evec[pj]
        for h in range(len(alphaj)):
            gamma += alphaj[i]*Polynomic(X[h],X_test,d,c)*alphaj[h]
    return gamma

def compute_gamma_Sigmoid_i(i,X_test,X,val,evec, alpha, c):
    gamma = 0
    for j in range(2):
        if j == 2:
            if max(val[:pj]) > max(val[pj+1:]):
                pj = np.argmax(val[:pj])
            else:
                pj = np.argmax(val[pj+1:])       
        pj = np.argmax(val)
        alphaj = evec[pj]
        for h in range(len(alphaj)):
            gamma += alphaj[i]*Sigmoid(X[h],X_test,alpha,c)*alphaj[h]
    return gamma
###########################################################
    
################### CROSS-VALIDATION ######################
def CrossValidation_ParameterH(kf , kernel, parameters, X): 
    RecError = []
    for train_index, test_index in kf.split(X):
        print('NEW GROUP')
        X_train= X[train_index]
        X_test = X[test_index]
        
        KM = computeKernelMatrix(kernel, X_train, parameters)
        val, evec = eigh(KM)
        
        for point in X_test:
            gamma = ComputeGamma(kernel, parameters, val, evec, point, X_train) #3000
            preimage = ComputePreImage(kernel, parameters, val, evec, X_train, gamma) #7500
            RecError.append(np.linalg.norm(preimage-point))
    
    print(sum(RecError)/len(RecError))
    return sum(RecError)/len(RecError)   


def OptimizeParameters(filename, K, kernel, grid_parameters):
    df = pd.read_csv(filename, index_col=0)
    df = df.to_numpy()
    
    X = df[:,[0,1]]
    #y = df[:,2]
    
    # Split dataset into groups
    kf = KFold(n_splits = K, shuffle = True, random_state = 1) # No randomness    
    
    first = True
    results = []
    
    for parameters in grid_parameters:
        print('NEW PARAMETERS')
        Rec_error = CrossValidation_ParameterH(kf, kernel, parameters, X)
        results.append({Rec_error: parameters})   
    return results
###########################################################

# 2 Code for plotting

In [ ]:
# FUNCTIONS FOR PLOTTING

from sklearn.decomposition import KernelPCA,PCA
import numpy as np
import matplotlib.pyplot as plt

def plot_PCA(X,reds,blues):
    pca = PCA()
    X_pca = pca.fit_transform(X)
    plt.figure()
    plt.plot(X_pca[reds, 0], X_pca[reds, 1], "ro")
    plt.plot(X_pca[blues, 0], X_pca[blues, 1], "bo")
    plt.title("Projection by PCA")
    plt.xlabel("1st principal component")
    plt.ylabel("2nd component")

def plot_KPCA(X,reds,blues,kernel, parameters):
    if kernel == 'rbf':
        kpca = KernelPCA(n_components=2, kernel='rbf', gamma = parameters['gamma'])
    elif kernel == 'polynomial':
        kpca = KernelPCA(n_components=2, kernel='polynomial', degree = parameters['d'], coef0 = parameters['c'])
    elif kernel == 'sigmoid':
        kpca = KernelPCA(n_components=2, kernel='polynomial', alpha = parameters['alpha'], coef0 = parameters['c'])
    X_kpca = kpca.fit_transform(X)
    plt.figure()
    plt.plot(X_kpca[reds, 0], X_kpca[reds, 1], "ro")
    plt.plot(X_kpca[blues, 0], X_kpca[blues, 1], "bo")
    plt.title("Projection by KPCA")
    plt.xlabel("1st principal component")
    plt.ylabel("2nd component")

# 3 TOY DATASETS

## 3.1 Blobs

### 3.1.1 Compute reconstruction error

#### RBF

In [ ]:
## INPUT ##
filename = 'Blobs_RE.csv'
kernel = 'rbf'
K = 15
grid_parameters = [{'sigma': 1/10},{'sigma': 1/5},{'sigma': 1/3}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

#### Polynomial

In [ ]:
## INPUT ##
filename = 'Blobs_RE.csv'
kernel = 'polynomial'
K = 15
grid_parameters = [{'d': 4, 'c':1},{'d': 3, 'c':2}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

#### Sigmoid

In [ ]:
## INPUT ##
filename = 'Blobs_RE.csv'
kernel = 'sigmoid'
K = 15
grid_parameters = [{'alpha': 3, 'c':2},{'alpha': 3, 'c':1}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

### 3.1.2 Plots 

In [ ]:
# READ DATA

df = pd.read_csv('Blobs.csv')
l = []
for i in df.index:
    l.append([df['x'][i],df['y'][i]])
X = np.array(l)
y = np.array(df['label'])
reds = y == 0
blues = y == 1

In [ ]:
kernel = 'rbf'
parameters = {'gamma' : 3}

plot_PCA(X,reds,blues)
plot_KPCA(X,reds,blues,kernel,parameters)

### 3.2 Circles

### 3.2.1 Compute reconstruction error

#### RBF

In [ ]:
## INPUT ##
filename = 'Circles.csv'
kernel = 'rbf'
K = 15
grid_parameters = [{'sigma': 1/10},{'sigma': 1/5},{'sigma': 1/3}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

#### Polynomial

In [ ]:
## INPUT ##
filename = 'Circles.csv'
kernel = 'polynomial'
K = 15
grid_parameters = [{'d': 4, 'c':1},{'d': 3, 'c':2}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

#### Sigmoid

In [ ]:
## INPUT ##
filename = 'Circles.csv'
kernel = 'Sigmoid'
K = 15
grid_parameters = [{'alpha': 3, 'c':2},{'alpha': 3, 'c':1}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

### 3.2.2 Plots 

In [ ]:
# READ DATA

df = pd.read_csv('Circles.csv')
l = []
for i in df.index:
    l.append([df['x'][i],df['y'][i]])
X = np.array(l)
y = np.array(df['label'])
reds = y == 0
blues = y == 1

In [ ]:
kernel = 'rbf'
parameters = {'gamma' : 3}

plot_PCA(X,reds,blues)
plot_KPCA(X,reds,blues,kernel,parameters)

## 2.3 Moons

### 3.3.1 Compute reconstruction error

#### RBF

In [ ]:
## INPUT ##
filename = 'Moons.csv'
kernel = 'rbf'
K = 15
grid_parameters = [{'sigma': 1/10},{'sigma': 1/5},{'sigma': 1/3}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

#### Polynomial

In [ ]:
## INPUT ##
filename = 'Moons.csv'
kernel = 'polynomial'
K = 15
grid_parameters = [{'d': 4, 'c':1},{'d': 3, 'c':2}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

#### Sigmoid

In [ ]:
## INPUT ##
filename = 'Moons.csv'
kernel = 'Sigmoid'
K = 15
grid_parameters = [{'alpha': 3, 'c':2},{'alpha': 3, 'c':1}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

### 3.2.2 Plots 

In [ ]:
# READ DATA

df = pd.read_csv('Moons.csv')
l = []
for i in df.index:
    l.append([df['x'][i],df['y'][i]])
X = np.array(l)
y = np.array(df['label'])
reds = y == 0
blues = y == 1

In [ ]:
kernel = 'rbf'
parameters = {'gamma' : 3}

plot_PCA(X,reds,blues)
plot_KPCA(X,reds,blues,kernel,parameters)

# 4.Code for four-D reconstruction error

The same code as described in point 1 but adapted for working with 4-D datasets

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from scipy.linalg import eigh
import random

from sklearn.model_selection import KFold
from sklearn.decomposition import KernelPCA

def iterateRBF(X_t,X,val,evec,gamma, sigma):
    num = 0
    den = 0 
    for i in range(len(X)):
        num += gamma[i]*Gaussian(X[i],X_t,sigma)*X[i]
        den += gamma[i]*Gaussian(X[i],X_t,sigma)
    if den == 0.0:
        return 0
    return num/den

def iteratePolynomic(X_t,X,val,evec,gamma, d, c):
    result = 0
    for i in range(len(X)):
        result += gamma[i]*pow((np.dot(X_t,X[i])+c)/(np.dot(X_t,X_t)+c),d-1)*X[i]
    return result


def Polynomic(x,z,d,c):
    return pow(np.dot(x,z)+c,d)

def Gaussian(x,z,sigma):
    return np.exp((-(np.linalg.norm(x-z)**2))/(2*sigma**2))

def Sigmoid(x,z,aplha,c):
    return np.tanh(aplha*np.dot(x,z)+c)

def vPolynomic(intr,d,c):
    return pow(intr,(1/d))-c

def vSigmoid(intr,alpha,c):
    return (np.arctanh(intr)-c)/alpha

def ComputePreImageRBF(X,val,evec, gamma, sigma):
    X_t = np.ones(len(X[0]))
    new_X_t = iterateRBF(X_t,X,val,evec,gamma, sigma) #1500
    i = 0
    k = 0
    while np.linalg.norm(new_X_t - X_t) > 0.001 and i < 500:
        X_t = new_X_t
        i += 1
        if i == 500 and k < 3:
            i = 0
            k += 1
            li = []
            for i in range(len(X[0])):
                li.append(random.uniform(-1, 1))
            X_t = np.array(li) 
        new_X_t = iterateRBF(X_t,X,val,evec,gamma, sigma)
    return new_X_t

def ComputePreImagePolynomic_odd(X,val,evec, gamma,d,c):
    e = np.array([[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,0.0,1.0]])
    preimage = np.array([0.0,0.0,0.0,0.0])
    for j in range(len(e)):
        intr = 0
        for i in range(len(X)):
            intr += gamma[i]*Polynomic(e[j],X[i],d,c)
        if intr < 0:
            intr = -intr
        preimage += vPolynomic(intr,d,c)*e[j]
    return preimage
        
def ComputePreImagePolynomic_even(X,val,evec, gamma, d, c):
    X_t = np.ones(len(X[0]))
    new_X_t = iteratePolynomic(X_t,X,val,evec,gamma, d, c)
    i = 0
    while np.linalg.norm(new_X_t - X_t) > 0.001 and i < 1000:
        X_t = new_X_t
        new_X_t = iteratePolynomic(X_t,X,val,evec,gamma, d, c)
        i += 1
    return new_X_t
        
def ComputePreImagePolynomic(X,val,evec, gamma, d, c):
    if d%2 == 0:
        return ComputePreImagePolynomic_even(X, val, evec, gamma, d, c)
    else:
        return ComputePreImagePolynomic_odd(X, val, evec, gamma, d, c)
        
def ComputePreImageSigmoid(X,val,evec, gamma, alpha, c):
    e = np.array([[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,0.0,1.0]])
    preimage = np.array([0.0,0.0,0.0,0.0])
    for j in range(len(e)):
        intr = 0
        for i in range(len(X)):
            intr += gamma[i]*Sigmoid(e[j],X[i],alpha,c)
        if intr <= -1:
            intr = -0.9
        if intr >= 1:
            intr = 0.9
        preimage += vSigmoid(intr,alpha,c)*e[j]
    return preimage

def compute_gamma_RBF_i(i,X_test,X,val,evec, sigma):
    gamma = 0
    for j in range(2):
        if j == 2:
            if max(val[:pj]) > max(val[pj+1:]):
                pj = np.argmax(val[:pj])
            else:
                pj = np.argmax(val[pj+1:])       
        pj = np.argmax(val)
        alphaj = evec[pj]
        for h in range(len(alphaj)):
            gamma += alphaj[i]*Gaussian(X[h],X_test,sigma)*alphaj[h]
    return gamma  

def compute_gamma_Polynomic_i(i,X_test,X,val,evec, d, c):
    gamma = 0
    for j in range(2):
        if j == 2:
            if max(val[:pj]) > max(val[pj+1:]):
                pj = np.argmax(val[:pj])
            else:
                pj = np.argmax(val[pj+1:])       
        pj = np.argmax(val)
        alphaj = evec[pj]
        for h in range(len(alphaj)):
            gamma += alphaj[i]*Polynomic(X[h],X_test,d,c)*alphaj[h]
    return gamma

def compute_gamma_Sigmoid_i(i,X_test,X,val,evec, alpha, c):
    gamma = 0
    for j in range(2):
        if j == 2:
            if max(val[:pj]) > max(val[pj+1:]):
                pj = np.argmax(val[:pj])
            else:
                pj = np.argmax(val[pj+1:])       
        pj = np.argmax(val)
        alphaj = evec[pj]
        for h in range(len(alphaj)):
            gamma += alphaj[i]*Sigmoid(X[h],X_test,alpha,c)*alphaj[h]
    return gamma

def ComputeGammaRBF(X_test, X, val, evec, sigma):
    gamma = []
    for i in range(len(X)):
        gamma.append(compute_gamma_RBF_i(i,X_test,X,val,evec,sigma))
    return gamma

def ComputeGammaPolynomic(X_test, X, val, evec, d, c):
    gamma = []
    for i in range(len(X)):
        gamma.append(compute_gamma_Polynomic_i(i,X_test,X,val,evec, d, c))
    return gamma

def ComputeGammaSigmoid(X_test, X, val, evec, aplha, c):
    gamma = []
    for i in range(len(X)):
        gamma.append(compute_gamma_Sigmoid_i(i,X_test,X,val,evec, aplha, c))
    return gamma

def Polynomic(x,z,d,c):
    return pow(np.dot(x,z)+c,d)

def Gaussian(x,z,sigma):
    return np.exp((-(np.linalg.norm(x-z)**2))/(2*sigma**2))

def Sigmoid(x,z,aplha,c):
    return np.tanh(aplha*np.dot(x,z)+c)


def GaussianMatrix(X,sigma):
    row,col=X.shape
    GassMatrix=np.zeros(shape=(row,row))
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            GassMatrix[i,j]=Gaussian(v_i.T,v_j.T,sigma)
            j+=1
        i+=1
    return GassMatrix

def SigmoidMatrix(X,alpha, c):
    row,col=X.shape
    SigmoidMatrix=np.zeros(shape=(row,row))
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            SigmoidMatrix[i,j]=Sigmoid(v_i.T,v_j.T,alpha, c)
            j+=1
        i+=1
    return SigmoidMatrix

def PolynomicalMatrix(X,d,c):
    row,col=X.shape
    PolMatrix=np.zeros(shape=(row,row))
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            PolMatrix[i,j]=Polynomic(v_i.T,v_j.T,d,c)
            j+=1
        i+=1
    return PolMatrix

def computeKernelMatrix(kernel, X, parameters):
    if kernel == 'rbf':
        return GaussianMatrix(X,parameters['sigma'])
    elif kernel == 'sigmoid':
        return SigmoidMatrix(X,parameters['alpha'],parameters['c'])
    elif kernel == 'polynomial':
        return PolynomicalMatrix(X,parameters['d'],parameters['c'])

def ComputeGamma(kernel, parameters, val, evec, point, X):
    if kernel == 'rbf':
        return ComputeGammaRBF(point, X, val, evec, parameters['sigma'])
    elif kernel == 'sigmoid':
        return ComputeGammaSigmoid(point, X, val, evec, parameters['alpha'],parameters['c'])
    elif kernel == 'polynomial':
        return ComputeGammaPolynomic(point, X, val, evec,parameters['d'],parameters['c'])

def ComputePreImage(kernel, parameters, val, evec, X, gamma):
    if kernel == 'rbf':
        return ComputePreImageRBF(X,val,evec, gamma, parameters['sigma'])
    elif kernel == 'sigmoid':
        return ComputePreImageSigmoid(X,val,evec, gamma, parameters['alpha'],parameters['c'])
    elif kernel == 'polynomial':
        return ComputePreImagePolynomic(X,val,evec, gamma, parameters['d'],parameters['c'])
    
def CrossValidation_ParameterH(kf , kernel, parameters, X): 
    RecError = []
    for train_index, test_index in kf.split(X):
        print('NEW GROUP')
        X_train= X[train_index]
        X_test = X[test_index]
        KM = computeKernelMatrix(kernel, X_train, parameters)
        val, evec = eigh(KM)
        for point in X_test:
            gamma = ComputeGamma(kernel, parameters, val, evec, point, X_train) #3000
            preimage = ComputePreImage(kernel, parameters, val, evec, X_train, gamma) #7500
            RecError.append(np.linalg.norm(preimage-point))
    
    print(sum(RecError)/len(RecError))
    return sum(RecError)/len(RecError)   


def OptimizeParameters(filename, K, kernel, grid_parameters):
    df = pd.read_csv(filename)
    l = []
    for i in df.index:
        l.append([df['x'][i],df['y'][i],df['w'][i],df['z'][i]])
    X = np.array(l)
    #y = df[:,2]
    # Split dataset into groups
    kf = KFold(n_splits = K, shuffle = True, random_state = 1) # No randomness    
    first = True
    results = []
    for parameters in grid_parameters:
        print('NEW PARAMETERS')
        Rec_error = CrossValidation_ParameterH(kf, kernel, parameters, X)
        results.append({Rec_error: parameters})   
    return results

# 5 MONEY DATASET

## 5.1 Compute reconstruction error

### RBF

In [ ]:
## INPUT ##
filename = 'money_RE.csv'
kernel = 'rbf'
K = 15
grid_parameters = [{'sigma': 1/10},{'sigma': 1/5},{'sigma': 1/3}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

### Polynomial

In [ ]:
## INPUT ##
filename = 'money_RE.csv'
kernel = 'polynomial'
K = 15
grid_parameters = [{'d': 4, 'c':1},{'d': 3, 'c':2}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

### Sigmoid

In [ ]:
## INPUT ##
filename = 'money_RE.csv'
kernel = 'Sigmoid'
K = 15
grid_parameters = [{'alpha': 3, 'c':2},{'alpha': 3, 'c':1}]
results = OptimizeParameters(filename, K, kernel, grid_parameters)

## 5.2 Plots

In [ ]:
# READ DATA

df = pd.read_csv('Money.csv')
l = []
for i in df.index:
    l.append([df['x'][i],df['y'][i],df['z'][i],df['w'][i]])
X = np.array(l)
y = np.array(df['label'])
reds = y == 0
blues = y == 1

In [ ]:
kernel = 'rbf'
parameters = {'gamma' : 3}

plot_PCA(X,reds,blues)
plot_KPCA(X,reds,blues,kernel,parameters)